In [1]:
# libraries importing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn import metrics
import numpy as np
import os

colors = ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf']

from algorithms.Vanilla_LSTM import Vanilla_LSTM
from process_data import process_data

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier

from feature_engine.selection import SmartCorrelatedSelection, DropConstantFeatures, DropDuplicateFeatures, DropFeatures, DropCorrelatedFeatures, SelectBySingleFeaturePerformance, RecursiveFeatureAddition
from feature_engine.outliers import Winsorizer

## Data loading

In [2]:
datasets = process_data()

valve1_X =  datasets["valve1_X"]
valve1_y = datasets["valve1_y"]
valve2_X = datasets["valve2_X"]
valve2_y = datasets["valve2_y"]
other_anomaly_X = datasets["other_anomaly_X"]
other_anomaly_y = datasets["other_anomaly_y"]

## Add data manipulation steps

__Drop duplicates__

In [3]:
print(f'valve 1 dataset shape: {valve1_X.shape}')
print(f'valve 2 dataset shape: {valve2_X.shape}')
print(f'other anomalies dataset shape: {other_anomaly_X.shape}')

valve1_duplicates = valve1_X.duplicated()
valve1_X = valve1_X.loc[~valve1_duplicates, :]
valve1_y = valve1_y.loc[~valve1_duplicates, :]

valve2_duplicates = valve2_X.duplicated()
valve2_X = valve2_X.loc[~valve2_duplicates, :]
valve2_y = valve2_y.loc[~valve2_duplicates, :]

other_anomaly_duplicates = other_anomaly_X.duplicated()
other_anomaly_X = other_anomaly_X.loc[~other_anomaly_duplicates, :]
other_anomaly_y = other_anomaly_y.loc[~other_anomaly_duplicates, :]

print('********************* Drop Duplicates *********************')
print(f'valve 1 dataset shape: {valve1_X.shape}')
print(f'valve 2 dataset shape: {valve2_X.shape}')
print(f'other anomalies dataset shape: {other_anomaly_X.shape}')

valve 1 dataset shape: (18162, 8)
valve 2 dataset shape: (4312, 8)
other anomalies dataset shape: (14985, 8)
********************* Drop Duplicates *********************
valve 1 dataset shape: (18162, 8)
valve 2 dataset shape: (4312, 8)
other anomalies dataset shape: (14542, 8)


__Winsorizer__

In [4]:
wz = Winsorizer(capping_method='quantiles', tail='both', fold=3)

In [5]:
# valve1_X = wz.fit_transform(valve1_X)
# valve2_X = wz.fit_transform(valve2_X)
# other_anomaly_X = wz.fit_transform(other_anomaly_X)

__Standard scaler__

In [6]:
sc = StandardScaler()

In [7]:
valve1_X = sc.fit_transform(valve1_X)
valve2_X = sc.fit_transform(valve2_X)
other_anomaly_X = sc.fit_transform(other_anomaly_X)

__PCA__

In [8]:
pca = PCA(n_components='mle', svd_solver='full')

In [9]:
# valve1_X = pca.fit_transform(valve1_X)
# valve2_X = pca.fit_transform(valve2_X)
# other_anomaly_X = pca.fit_transform(other_anomaly_X)

__RFE based on SVM__

In [10]:
estimator = SVR(kernel="linear")
rfe = RFE(estimator, n_features_to_select=5, step=1)

In [11]:
# valve1_X = rfe.fit_transform(valve1_X, valve1_y.anomaly)
# valve2_X = rfe.fit_transform(valve2_X, valve2_y.anomaly)
# other_anomaly_X = rfe.fit_transform(other_anomaly_X, other_anomaly_y.anomaly)

__Feature selection by single feature performance using random forest estimator__

In [12]:
sfp = SelectBySingleFeaturePerformance(
                    RandomForestClassifier(random_state=42),
                    cv=2)

In [13]:
# valve1_X = sfp.fit_transform(valve1_X, valve1_y.anomaly)
# valve2_X = sfp.fit_transform(valve2_X, valve2_y.anomaly)
# other_anomaly_X = sfp.fit_transform(other_anomaly_X, other_anomaly_y.anomaly)

__Feature selection by information value__

In [14]:
rfa = RecursiveFeatureAddition(RandomForestClassifier(random_state=42), cv=2)

In [15]:
# valve1_X = rfa.fit_transform(valve1_X, valve1_y.anomaly)
# valve2_X = rfa.fit_transform(valve2_X, valve2_y.anomaly)
# other_anomaly_X = rfa.fit_transform(other_anomaly_X, other_anomaly_y.anomaly)

__Smart correlated features__

In [16]:
scs = SmartCorrelatedSelection(threshold=0.8)

In [17]:
# # print(f'valve 1 dataset number of columns: {len(valve1_X.columns)}')
# # print(f'valve 2 dataset number of columns: {len(valve2_X.columns)}')
# # print(f'other animalies dataset number of columns: {len(other_anomaly_X.columns)}')

# valve1_X = scs.fit_transform(valve1_X)
# valve2_X = scs.fit_transform(valve2_X)
# other_anomaly_X = scs.fit_transform(other_anomaly_X)

# # print('********************* Drop correlated columns *********************')
# # print(f'valve 1 dataset number of columns: {len(valve1_X.columns)}')
# # print(f'valve 2 dataset number of columns: {len(valve2_X.columns)}')
# # print(f'other animalies dataset number of columns: {len(other_anomaly_X.columns)}')

In [18]:
# hyperparameters selection
N_STEPS = 5
EPOCHS = 10
BATCH_SIZE = 32
VAL_SPLIT = 0.2
Qs = np.arange(0.55, 0.90, 0.05) # quantile for upper control limit (UCL) selection
PARAMS = [N_STEPS, EPOCHS, BATCH_SIZE, VAL_SPLIT]
model = Vanilla_LSTM(PARAMS)

In [19]:
def test_train_split(df_X, df_y):
    size_train = int(df_X.shape[0]*0.8)
    size_test = df_X.shape[0] - size_train
    x_train = df_X[:size_train]
    y_train = df_y[:size_train].anomaly
    x_test = df_X[-size_test:]
    y_test = df_y[-size_test:].anomaly
    return x_train, y_train, x_test, y_test

In [20]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

#### Test model for valve 1

In [21]:
results_valve1 = pd.DataFrame(columns = ['Q', 'TPR', 'TNR', 'PPV', 'NPV', 'FPR', 'FNR', 'FDR', 'ACC'])

x_train, y_train, x_test, y_test = test_train_split(valve1_X, valve1_y)
# x_train_steps, y_train_steps = split_sequences(np.array([row.values for i, row in x_train.iterrows()]), N_STEPS)
# x_test_steps, y_test_steps = split_sequences(np.array([row.values for i, row in x_test.iterrows()]), N_STEPS)
x_train_steps, y_train_steps = split_sequences(x_train, N_STEPS)
x_test_steps, y_test_steps = split_sequences(x_test, N_STEPS)

model.fit(x_train_steps,y_train_steps)

for Q in Qs:

    # results predicting
    residuals_train = pd.DataFrame(y_train_steps - model.predict(x_train_steps)).abs().sum(axis=1)
    UCL = residuals_train.quantile(Q)

#     # train predicting
#     lstm_residuals = pd.DataFrame(y_train_steps - model.predict(x_train_steps)).abs().sum(axis=1)
#     yhat_train = pd.Series((lstm_residuals > UCL).astype(int).values, 
#                                 index=x_train[N_STEPS:].index).fillna(0)

#     # test prediction
#     lstm_residuals = pd.DataFrame(y_test_steps - model.predict(x_test_steps)).abs().sum(axis=1)
#     yhat_test = pd.Series((lstm_residuals > UCL).astype(int).values, 
#                                 index=x_test[N_STEPS:].index).fillna(0)
    # train prediction
    lstm_residuals = pd.DataFrame(y_train_steps - model.predict(x_train_steps)).abs().sum(axis=1)
    yhat_train = pd.Series((lstm_residuals > UCL).astype(int).values, 
                        index=np.arange(len(x_train[N_STEPS:]))).fillna(0)
    #test prediction
    lstm_residuals = pd.DataFrame(y_test_steps - model.predict(x_test_steps)).abs().sum(axis=1)
    yhat_test = pd.Series((lstm_residuals > UCL).astype(int).values, 
                        index=np.arange(len(x_test[N_STEPS:]))).fillna(0)

    conf_matrix = metrics.confusion_matrix(y_test[N_STEPS:], yhat_test)

    TN, FP, FN, TP = conf_matrix.ravel()

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate FAR false alarm rate
    FPR = FP/(FP+TN)
    # False negative rate MAR missing alarm rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)

    row = dict(Q = Q,
            TPR = TPR,
            TNR = TNR,
            PPV = PPV,
            NPV = NPV,
            FPR = FPR,
            FNR = FNR,
            FDR = FDR,
            ACC = ACC)

    results_valve1 = pd.concat([results_valve1, pd.DataFrame(row, index = [0])], ignore_index = True)

123/454 [=======>......................] - ETA: 0s

/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_82704/849091213.py:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_valve1 = pd.concat([results_valve1, pd.DataFrame(row, index = [0])], ignore_index = True)


114/114 [==============================] - 0s 1ms/step


In [22]:
for i, col in enumerate(results_valve1.columns[1:]):
    fig = go.Figure()

    fig.add_trace(go.Scatter(mode='lines+text', x=results_valve1.Q, y=results_valve1[f'{col}'],
                            marker=dict(color=colors[i]),
                            texttemplate='%{y:.2f}', textposition='top center',
                            textfont=dict(color=colors[i], size=12),
                            name=f'{col}',
                            showlegend=True)
                    )

    fig.update_layout(height=400,width=900, template='plotly_white',
                    title=dict(text=f'{col} with different Q values', font=dict(size=18), x=.5, y=.95),
                    yaxis=dict(title=f'{col}', side='left', showgrid=True,),
                    xaxis=dict(title='Q', showgrid=False),
                    legend=dict(orientation="h", yanchor="bottom", y=1, x=0.5, xanchor="center"),
                    )

    fig.show()

#### Test model for valve 2

In [23]:
results_valve2 = pd.DataFrame(columns = ['Q', 'TPR', 'TNR', 'PPV', 'NPV', 'FPR', 'FNR', 'FDR', 'ACC'])

x_train, y_train, x_test, y_test = test_train_split(valve2_X, valve2_y)
# x_train_steps, y_train_steps = split_sequences(np.array([row.values for i, row in x_train.iterrows()]), N_STEPS)
# x_test_steps, y_test_steps = split_sequences(np.array([row.values for i, row in x_test.iterrows()]), N_STEPS)
x_train_steps, y_train_steps = split_sequences(x_train, N_STEPS)
x_test_steps, y_test_steps = split_sequences(x_test, N_STEPS)

model.fit(x_train_steps,y_train_steps)

for Q in Qs:

    # results predicting
    residuals_train = pd.DataFrame(y_train_steps - model.predict(x_train_steps)).abs().sum(axis=1)
    UCL = residuals_train.quantile(Q)

#     # train predicting
#     lstm_residuals = pd.DataFrame(y_train_steps - model.predict(x_train_steps)).abs().sum(axis=1)
#     yhat_train = pd.Series((lstm_residuals > UCL).astype(int).values, 
#                                 index=x_train[N_STEPS:].index).fillna(0)

#     # test prediction
#     lstm_residuals = pd.DataFrame(y_test_steps - model.predict(x_test_steps)).abs().sum(axis=1)
#     yhat_test = pd.Series((lstm_residuals > UCL).astype(int).values, 
#                                 index=x_test[N_STEPS:].index).fillna(0)
    # train prediction
    lstm_residuals = pd.DataFrame(y_train_steps - model.predict(x_train_steps)).abs().sum(axis=1)
    yhat_train = pd.Series((lstm_residuals > UCL).astype(int).values, 
                        index=np.arange(len(x_train[N_STEPS:]))).fillna(0)
    #test prediction
    lstm_residuals = pd.DataFrame(y_test_steps - model.predict(x_test_steps)).abs().sum(axis=1)
    yhat_test = pd.Series((lstm_residuals > UCL).astype(int).values, 
                        index=np.arange(len(x_test[N_STEPS:]))).fillna(0)

    conf_matrix = metrics.confusion_matrix(y_test[N_STEPS:], yhat_test)

    TN, FP, FN, TP = conf_matrix.ravel()

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate FAR false alarm rate
    FPR = FP/(FP+TN)
    # False negative rate MAR missing alarm rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)

    row = dict(Q = Q,
            TPR = TPR,
            TNR = TNR,
            PPV = PPV,
            NPV = NPV,
            FPR = FPR,
            FNR = FNR,
            FDR = FDR,
            ACC = ACC)

    results_valve2 = pd.concat([results_valve2, pd.DataFrame(row, index = [0])], ignore_index = True)

  1/108 [..............................] - ETA: 1s

/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_82704/1703976981.py:66: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



27/27 [==============================] - 0s 1ms/step


In [24]:
for i, col in enumerate(results_valve2.columns[1:]):
    fig = go.Figure()

    fig.add_trace(go.Scatter(mode='lines+text', x=results_valve2.Q, y=results_valve2[f'{col}'],
                            marker=dict(color=colors[i]),
                            texttemplate='%{y:.2f}', textposition='top center',
                            textfont=dict(color=colors[i], size=12),
                            name=f'{col}',
                            showlegend=True)
                    )

    fig.update_layout(height=400,width=900, template='plotly_white',
                    title=dict(text=f'{col} with different Q values', font=dict(size=18), x=.5, y=.95),
                    yaxis=dict(title=f'{col}', side='left', showgrid=True,),
                    xaxis=dict(title='Q', showgrid=False),
                    legend=dict(orientation="h", yanchor="bottom", y=1, x=0.5, xanchor="center"),
                    )

    fig.show()

#### Test model for other anomalies

In [25]:
results_other_anomaly = pd.DataFrame(columns = ['Q', 'TPR', 'TNR', 'PPV', 'NPV', 'FPR', 'FNR', 'FDR', 'ACC'])

x_train, y_train, x_test, y_test = test_train_split(other_anomaly_X, other_anomaly_y)
# x_train_steps, y_train_steps = split_sequences(np.array([row.values for i, row in x_train.iterrows()]), N_STEPS)
# x_test_steps, y_test_steps = split_sequences(np.array([row.values for i, row in x_test.iterrows()]), N_STEPS)
x_train_steps, y_train_steps = split_sequences(x_train, N_STEPS)
x_test_steps, y_test_steps = split_sequences(x_test, N_STEPS)

model.fit(x_train_steps,y_train_steps)

for Q in Qs:

    # results predicting
    residuals_train = pd.DataFrame(y_train_steps - model.predict(x_train_steps)).abs().sum(axis=1)
    UCL = residuals_train.quantile(Q)

#     # train predicting
#     lstm_residuals = pd.DataFrame(y_train_steps - model.predict(x_train_steps)).abs().sum(axis=1)
#     yhat_train = pd.Series((lstm_residuals > UCL).astype(int).values, 
#                                 index=x_train[N_STEPS:].index).fillna(0)

#     # test prediction
#     lstm_residuals = pd.DataFrame(y_test_steps - model.predict(x_test_steps)).abs().sum(axis=1)
#     yhat_test = pd.Series((lstm_residuals > UCL).astype(int).values, 
#                                 index=x_test[N_STEPS:].index).fillna(0)
    # train prediction
    lstm_residuals = pd.DataFrame(y_train_steps - model.predict(x_train_steps)).abs().sum(axis=1)
    yhat_train = pd.Series((lstm_residuals > UCL).astype(int).values, 
                        index=np.arange(len(x_train[N_STEPS:]))).fillna(0)
    #test prediction
    lstm_residuals = pd.DataFrame(y_test_steps - model.predict(x_test_steps)).abs().sum(axis=1)
    yhat_test = pd.Series((lstm_residuals > UCL).astype(int).values, 
                        index=np.arange(len(x_test[N_STEPS:]))).fillna(0)

    conf_matrix = metrics.confusion_matrix(y_test[N_STEPS:], yhat_test)

    TN, FP, FN, TP = conf_matrix.ravel()

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate FAR false alarm rate
    FPR = FP/(FP+TN)
    # False negative rate MAR missing alarm rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)

    row = dict(Q = Q,
            TPR = TPR,
            TNR = TNR,
            PPV = PPV,
            NPV = NPV,
            FPR = FPR,
            FNR = FNR,
            FDR = FDR,
            ACC = ACC)

    results_other_anomaly = pd.concat([results_other_anomaly, pd.DataFrame(row, index = [0])], ignore_index = True)

124/364 [=========>....................] - ETA: 0s

/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_82704/1854636055.py:46: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_82704/1854636055.py:66: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



123/364 [=========>....................] - ETA: 0s

/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_82704/1854636055.py:46: RuntimeWarning:

invalid value encountered in scalar divide



126/364 [=========>....................] - ETA: 0s

...

/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_82704/1854636055.py:46: RuntimeWarning:

invalid value encountered in scalar divide



121/364 [========>.....................] - ETA: 0s

/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_82704/1854636055.py:46: RuntimeWarning:

invalid value encountered in scalar divide



91/91 [==============================] - 0s 1ms/step


/var/folders/zz/t2f5q7kj0bjdkxwz1v713q7w0000gn/T/ipykernel_82704/1854636055.py:46: RuntimeWarning:

invalid value encountered in scalar divide



In [26]:
for i, col in enumerate(results_other_anomaly.columns[1:]):
    fig = go.Figure()

    fig.add_trace(go.Scatter(mode='lines+text', x=results_other_anomaly.Q, y=results_other_anomaly[f'{col}'],
                            marker=dict(color=colors[i]),
                            texttemplate='%{y:.2f}', textposition='top center',
                            textfont=dict(color=colors[i], size=12),
                            name=f'{col}',
                            showlegend=True)
                    )

    fig.update_layout(height=400,width=900, template='plotly_white',
                    title=dict(text=f'{col} with different Q values', font=dict(size=18), x=.5, y=.95),
                    yaxis=dict(title=f'{col}', side='left', showgrid=True,),
                    xaxis=dict(title='Q', showgrid=False),
                    legend=dict(orientation="h", yanchor="bottom", y=1, x=0.5, xanchor="center"),
                    )

    fig.show()